In [264]:
# colors = {'1':'r', '2':'g', 'Iris-virginica':'b'}

# # fig, ax = plt.subplots()
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d') 
# for i in range(len(X_train)):
#      ax.scatter(X_train[i][5] ,X_train[i][1], X_train[i][2],color=colors[Y_train[i]] , s=50, alpha=0.6, edgecolors='w')


In [265]:
# library 😚

import sklearn.datasets as datasets
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

from scipy.io.arff import loadarff

import numpy as np
from sklearn import metrics

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import collections

In [266]:
# config 😚
path_to_datasets = '/home/farzad/Desktop/semiWithTree/originDataset/'
dataset_name = 'bupa'


dataset_path  = path_to_datasets + dataset_name
base_classifier = DecisionTreeClassifier
random_state = 0
min_samples_leaf=3

In [267]:
def divide_xy(test_data):
    # assert : class = last atr 😚
    x_test = test_data.values[:, 0:-1]
    y_test = (test_data.values[:, -1]).astype('int')
    
    return x_test,y_test

def read_data(dataset_path) :
    
    train_raw_data = loadarff(dataset_path+'/train.arff')
    test_raw_data = loadarff(dataset_path+'/test.arff')
    
    train_data = pd.DataFrame(train_raw_data[0])
    test_data = pd.DataFrame(test_raw_data[0])
    
    train_data['Class'] = train_data['Class'].astype(int)
    test_data['Class'] = test_data['Class'].astype(int)
    
    return train_data,test_data


def get_rate_p(train_y) : 
    
    counter=collections.Counter(train_y)
    tuple_list_pn = counter.most_common()
    
    return tuple_list_pn[0][1]/(tuple_list_pn[0][1]+tuple_list_pn[1][1]) , tuple_list_pn
    

In [268]:
# int([1 , 2.0])
# # int(b'1')

In [269]:
# train_data,test_data = read_data(dataset_path)
# # (train_x,train_y),(test_x , test_y) 
# counter=collections.Counter(train_y)
# a=counter.most_common()
# a[0][1]/a[1][1]

In [270]:
def split_trainset(train_data) :
    
    labeled , unlabeled = [],[]
    
    size_dataset = len(train_data)
    train_x,train_y = divide_xy(train_data)
    
    rate_p , tuple_list_pn = get_rate_p(train_y)
    
    size_labeled_data = round(0.1 * size_dataset)
    size_unlabeled_data = size_dataset - size_labeled_data
    
    size_labeled_p_data = round(rate_p*size_labeled_data)
    size_labeled_n_data = size_labeled_data - size_labeled_p_data
    
    labeled_index = []
    unlabeled_index = []
    selected_pl = 0
    selected_nl = 0
    
    for i,cls in enumerate(train_y):
        # if data point class's == 0 😚
        if cls == tuple_list_pn[0][0] :
            if selected_pl < size_labeled_p_data :
                labeled_index.append(i)
                selected_pl+=1
            else :
                unlabeled_index.append(i)
        else :
            if selected_nl < size_labeled_n_data :
                labeled_index.append(i)
                selected_nl+=1
            else :
                unlabeled_index.append(i)
                
    for i in labeled_index:
        labeled.append(train_data.values[i])
    
    for i in unlabeled_index:
        unlabeled.append(train_data.values[i])
    
#     print(size_dataset , size_labeled_data , size_unlabeled_data)
#     print(rate_p , tuple_list_pn)
#     print(size_labeled_p_data , size_labeled_n_data)
#     print(selected_pl/(selected_pl+selected_nl),selected_pl, selected_nl)
    
    
    return pd.DataFrame(labeled,columns=train_data.columns),pd.DataFrame(unlabeled,columns=train_data.columns),rate_p,tuple_list_pn

In [271]:
def evaluate_classifier(base_classifier, labeled_data, test_data):

    labeled_x,labeled_y = divide_xy(labeled_data)
    test_x,test_y = divide_xy(test_data)

    # dtree = DecisionTreeClassifier(criterion="entropy", max_depth=3)
    dtree=base_classifier(random_state = random_state, min_samples_leaf=min_samples_leaf)
    dtree.fit(labeled_x,labeled_y)
    
    dot_data = StringIO()
    export_graphviz(dtree, out_file=dot_data,filled=True, rounded=True,special_characters=True)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
    img = Image(graph.create_png())
    
    y_pred = dtree.predict(test_x)
    
    accuracy = metrics.accuracy_score(test_y, y_pred)
    
    return accuracy , img


In [272]:
# train_data,test_data = read_data(dataset_path)
# labeled_data , unlabeled_data , rate_p,tuple_list_pn = split_trainset(train_data)
# # (train_x,train_y),(test_x , test_y) 
# unlabeled_data.at[1, 'Class'] = 0
# pd.concat([labeled_data , pd.DataFrame([unlabeled_data.values[1]] , columns=labeled_data.columns)] , ignore_index=True)
# # labeled_data
# # unlabeled_data


In [273]:
def confidency(name , DTclassifier , labeled_data , unlabeled_data , i , confidence) :

    test_x,test_y = divide_xy(unlabeled_data)
    is_confident = False
    lbl = DTclassifier.predict([test_x[i]])
    if name == 'Probability' :
        i_confidence = DTclassifier.predict_proba([test_x[i]])
        if max(i_confidence[0]) > confidence :
            is_confident = True
    
    return is_confident , lbl


def selection_metric(labeled_data,unlabeled_data ,rate_p,tuple_list_pn , confidence,selection_rate , confidence_method_name) :
    
    labeled_x,labeled_y = divide_xy(labeled_data)
    unlabeled_x,unlabeled_y = divide_xy(unlabeled_data)

    # dtree = DecisionTreeClassifier(criterion="entropy", max_depth=3)
    DTclassifier = base_classifier(random_state = random_state, min_samples_leaf=min_samples_leaf)
    DTclassifier.fit(labeled_x,labeled_y)
    
    removed_selected_data = unlabeled_data.copy()
    total_selected_labeling = pd.DataFrame(columns=labeled_data.columns)
    selected_labeling = pd.DataFrame(columns=labeled_data.columns)
    selected_index = []
    selected_y = []
    
    
    for i  in range(len(removed_selected_data)) :
        is_confident , lbl = confidency(confidence_method_name , DTclassifier , labeled_data ,
                                        removed_selected_data , i , confidence)
        if is_confident: 
            selected_index.append(i)
            selected_y.append(lbl[0])
            # set class
            removed_selected_data.at[i, 'Class'] = lbl[0]
            
    #should be constant rate 
#     print(np.array(selected_y))
    new_rate_p,new_tuple_list_pn = get_rate_p(np.array(selected_y))
    selected_index_p = []
    selected_index_n = []
    size_selected  = selection_rate * len(labeled_data)
    size_select_p = size_selected * rate_p
    
    if(size_select_p > new_tuple_list_pn[0][1] ) : 
        raise Exception("size_select_p > new_tuple_list_pn[0][1] \n change selection_rate ") 
        
    p = new_tuple_list_pn[0][0]
    for i in range(len(selected_y)):
        if size_selected == 0 :
            break
        if selected_y[i] == p :
            selected_index_p.append(i)
        else :
            selected_index_n.append(i)
            
        size_selected-=1
        
        
    for i in range(len(selected_index_p)):
        selected_labeling =pd.concat([selected_labeling ,
                                      pd.DataFrame([removed_selected_data.values[i]],
                                                   columns=labeled_data.columns)],    ignore_index=True)
    for i in range(len(selected_index_n)):
        selected_labeling =pd.concat([selected_labeling ,
                                      pd.DataFrame([removed_selected_data.values[i]],
                                                   columns=labeled_data.columns)],    ignore_index=True)

    removed_selected_data.drop(removed_selected_data.index[selected_index])
    
    total_selected_labeling = pd.concat([labeled_data ,selected_labeling],ignore_index=True)
    
    return total_selected_labeling,removed_selected_data


def self_labeling(labeled_data , unlabeled_data , iteration , rate_p,tuple_list_pn , confidence,selection_rate,confidence_method_name):

    
    labeled_unlabel_data = labeled_data.copy()
    removed_selected_data = unlabeled_data.copy()
    
    while iteration:
        
        selected_labeling,removed_selected_data = selection_metric(labeled_unlabel_data,removed_selected_data,
                                                                   rate_p,tuple_list_pn ,
                                                                   confidence,selection_rate,
                                                                   confidence_method_name)
        labeled_unlabel_data = pd.concat([labeled_unlabel_data , selected_labeling])
        
        print('iteration:' , iteration , ' , selected_labeling:' , len(selected_labeling))
        iteration-=1
        
    return labeled_unlabel_data
        
        

In [274]:

def main():
    
    train_data,test_data = read_data(dataset_path)
    train_x,train_y = divide_xy(train_data)
    test_x , test_y = divide_xy(test_data)
    
    labeled_data,unlabeled_data , rate_p,tuple_list_pn = split_trainset(train_data)
    
    a1 , img1 = evaluate_classifier(base_classifier , labeled_data  , test_data)
    
    total_labeled_data = self_labeling(labeled_data , unlabeled_data , 1, 
                                       rate_p,tuple_list_pn ,
                                       confidence=0.9,selection_rate = 1,confidence_method_name='Probability')
    
    a2 , img2 = evaluate_classifier(base_classifier , total_labeled_data  , test_data)
    
    print(a1 , a2)
    

In [275]:
train_data,test_data = read_data(dataset_path)
labeled_data , unlabeled_data , rate_p,tuple_list_pn = split_trainset(train_data)
# (train_x,train_y),(test_x , test_y) 

# a , b = evaluate_classifier(base_classifier , labeled_data  , test_data)
# print(a)
# # b
# # labeled_data.at[0, labeled_data.columns[-1]] = 0
# for i  in range(len(labeled_data)) :
# #     print(labeled_data.loc[i])
#     break
    
# # labeled_data.add(labeled_data.loc[0])
# labeled_data.at[i, 'Class']
# # labeled_data.loc[]
# # train_data.columns
# # test_data.columns


labeled_x,labeled_y = divide_xy(labeled_data)
unlabeled_x,unlabeled_y = divide_xy(unlabeled_data)

# dtree = DecisionTreeClassifier(criterion="entropy", max_depth=3)
DTclassifier = base_classifier(random_state = random_state , min_samples_leaf=min_samples_leaf)
DTclassifier.fit(unlabeled_x,unlabeled_y)

i=5
DTclassifier.predict([labeled_x[i]]) , labeled_y[i] , DTclassifier.predict_proba([labeled_x[i]])

(array([2]), 1, array([[0., 1.]]))

In [276]:
main()

iteration: 1  , selected_labeling: 46
0.6106194690265486 0.5929203539823009


In [277]:
get_rate_p(np.array([1 , 2]))

(0.5, [(1, 1), (2, 1)])